In [1]:
import pandas as pd
import requests

In [2]:
response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=1')

In [3]:
df=pd.DataFrame(response.json()['results'])[['title','genre_ids','overview']]
df.head()

,title,genre_ids,overview
0,Impossible Things,"[10751, 18]",A widow who is tormented by the memory of her ...
1,The Shawshank Redemption,"[18, 80]",Framed in the 1940s for the double murder of h...
2,Dilwale Dulhania Le Jayenge,"[35, 18, 10749]","Raj is a rich, carefree, happy-go-lucky second..."
3,The Godfather,"[18, 80]","Spanning the years 1945 to 1955, a chronicle o..."
4,Schindler's List,"[18, 36, 10752]",The true story of how businessman Oskar Schind...


In [4]:
df = pd.DataFrame()

In [5]:
for i in range(1,472):
    response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={}'.format(i))
    temp_df=pd.DataFrame(response.json()['results'])[['title','genre_ids','overview']]
    df = df.append(temp_df,ignore_index=True)



In [6]:
df.shape

(9420, 3)

In [7]:
genre = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')

In [8]:
genreData = genre.json()['genres']
gh = pd.DataFrame(genreData)
gh.set_index('id',inplace=True)
gh

,name
id,
28,Action
12,Adventure
16,Animation
35,Comedy
80,Crime
99,Documentary
18,Drama
10751,Family
14,Fantasy


In [9]:
gh.values[2]

array(['Animation'], dtype=object)

In [10]:
newMoviedf = pd.DataFrame({"genre":[]})
for i in df.index: #8560
    movieData = ""
    for genId in df['genre_ids'][i]: # 22170
        movieData+=gh['name'][genId]+", "
    newMdf = pd.DataFrame({"genre":[movieData]})
    newMoviedf = newMoviedf.append(newMdf,ignore_index = True)
newMoviedf.head()

,genre
0,"Family, Drama,"
1,"Drama, Crime,"
2,"Comedy, Drama, Romance,"
3,"Drama, Crime,"
4,"Drama, History, War,"


In [11]:
result = pd.concat([df, newMoviedf], axis=1)
result.drop(columns='genre_ids',axis=1,inplace=True)

In [12]:
result.head()

,title,overview,genre
0,Impossible Things,A widow who is tormented by the memory of her ...,"Family, Drama,"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama, Crime,"
2,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy, Drama, Romance,"
3,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama, Crime,"
4,Schindler's List,The true story of how businessman Oskar Schind...,"Drama, History, War,"


In [13]:
result['overview']=result['overview'].str.lower()


In [14]:
import re
def remove_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)
for i in df.index:
    result['overview'][i]= remove_tags(result['overview'][i])                                   


In [15]:
result.head()

,title,overview,genre
0,Impossible Things,a widow who is tormented by the memory of her ...,"Family, Drama,"
1,The Shawshank Redemption,framed in the 1940s for the double murder of h...,"Drama, Crime,"
2,Dilwale Dulhania Le Jayenge,"raj is a rich, carefree, happy-go-lucky second...","Comedy, Drama, Romance,"
3,The Godfather,"spanning the years 1945 to 1955, a chronicle o...","Drama, Crime,"
4,Schindler's List,the true story of how businessman oskar schind...,"Drama, History, War,"


In [16]:
def remove_urls(vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)
for i in df.index:
    result['overview'][i]= remove_urls(result['overview'][i])                                   

In [17]:
import string

def punk(a_string):
    new_string = a_string.translate(str.maketrans('', '', string.punctuation))
    return new_string
for i in result.index:
    result['overview'][i]= punk(result['overview'][i])                                   

In [18]:
result['overview'][6]

'with dreams of diving abroad tsuneo gets a job assisting josee an artist whose imagination takes her far beyond her wheelchair but when the tide turns against them they push each other to places they never thought possible and inspire a love fit for a storybook'

In [19]:
url = "https://raw.githubusercontent.com/AmarisAI/sms-slang-translator/master/slang.txt"
col =["A","B"]
gh = pd.read_csv(url,names=col,on_bad_lines='skip',sep='=')
gh = gh.set_index('A')
gh.to_dict(orient='dict')
gh =gh.B
gh

A
AFAIK       As Far As I Know
AFK       Away From Keyboard
ASAP     As Soon As Possible
ATK          At The Keyboard
ATM            At The Moment
                ...         
WTF            What The F...
WTG               Way To Go!
WUF      Where Are You From?
W8                   Wait...
7K           Sick:-D Laugher
Name: B, Length: 68, dtype: object

In [20]:
gh['AFK']

'Away From Keyboard'

In [21]:
def chat(short):
    newshort = []
    for w in short.split():
        if w.upper() in gh:
            newshort.append(gh[w.upper()])
        else:
            newshort.append(w)
    return " ".join(newshort)

for i in result.index:
    result['overview'][i]= chat(result['overview'][i])                                   
    



In [22]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [23]:
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)
for i in result.index:
    result['overview'][i]= remove_stopwords(result['overview'][i])                                   

In [26]:
from nltk.stem import PorterStemmer

In [28]:
ps = PorterStemmer()
def newstem(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [29]:
newstem(result.overview[1])

'frame 1940 doubl murder wife lover upstand banker andi dufresn begin new life shawshank prison put account skill work amor warden long stretch prison dufresn come admir inmat includ older prison name red integr unquench sens hope'

In [31]:
for j in result.index:
    test = result['overview'][j] 
    result['overview'][j]=newstem(test) 

In [32]:
result.overview.head()

0    widow torment memori abus husband befriend you...
1    frame 1940 doubl murder wife lover upstand ban...
2    raj rich carefre happygolucki second gener nri...
3    span year 1945 1955 chronicl fiction italianam...
4    true stori businessman oskar schindler save th...
Name: overview, dtype: object